In [288]:
import numpy as np
c = np.array([10, 12, 12])
A = np.array([[1, 2, 2], [2, 1, 2], [2, 2, 1]])
b = np.array([20, 20, 20])

In [289]:
c = np.array([2, 1])
A = np.array([[1, 1], [-1, 1], [6, 2]])
b = np.array([5, 0, 21])

In [290]:
c = np.array([2, 1])
A = np.array([[1, 1], [-1, 1], [6, 2]])
b = np.array([5, 0, 21])

In [291]:
def dual_simplex(T,basis):
    print("Entered Dual Simplex")
    m,n = T.shape
    n = n-1
    m = m-1
    while True:
        if np.all(T[:-1,-1] >= -tol):
            x = np.zeros(n)
            x[basis] = T[:-1, -1]
            return T,basis,x
        
        for it in range(m):
            if T[it,-1] < 0:
                l = it
                
                
                if np.all(T[l,:-1] >= -tol):
                    return None, None, None

                min_index = -1
                min_ratio = 1e9
                for rit in range(n):
                    if(T[l,rit] < 0):
                        if T[-1,rit]/abs(T[l,rit]) < min_ratio:
                            min_index = rit
                            min_ratio = T[-1,rit]/abs(T[l,rit])
                            
                j = min_index
        
                if min_index == -1:
                    # unbounded solution
                    return None, None,None
                
                
                # step 3: perform pivot
                basis[l] = j
                
                T[l,:] /= T[l,j]
                for k in range(m+1):
                    if k != l:
                        T[k,:] -= T[k,j] * T[l,:]
                                   
                break        
        # j = np.argmin(T[-1,:-1])

In [292]:
tol = 1e-7

def tableau_phase1(T,n,m,basis):
    while True:
        j = np.argmin(T[-1,:-1])
        if T[-1,j] >= -tol:
            # optimal solution found
            x = np.zeros(n)
            # TODO
            # x[basis < n] = T[basis < n, -1]
            return T,basis, x
        # step 2: select leaving variable
        # T[:-1,-1]/T[:-1,j]
        
        min_index = -1
        min_ratio = 1e9
        for it in range(m):
            if T[it,j] > 0:
                if T[it,-1]/T[it,j] < min_ratio:
                    min_index = it
                    min_ratio = T[it,-1]/T[it,j] 
        
        # i = np.argmin(T[:-1,-1]/T[:-1,j]) #Doubt about correctness of this
        
        i = min_index
        
        if min_index == -1:
            # unbounded solution
            return None, None,None
        
        
        # step 3: perform pivot
        basis[i] = j
        
        T[i,:] /= T[i,j]
        for k in range(m+1):
            if k != i:
                T[k,:] -= T[k,j] * T[i,:]


def tableau_phase2(T,n,m,basis):
    while True:
        j = np.argmin(T[-1,:-1])
        if T[-1,j] >= -tol:
            # optimal solution found
            x = np.zeros(n)
            x[basis] = T[:-1, -1]
            return T,basis, x
        # step 2: select leaving variable
        # T[:-1,-1]/T[:-1,j]
        
        min_index = -1
        min_ratio = 1e9
        for it in range(m):
            if T[it,j] > 0:
                if T[it,-1]/T[it,j] < min_ratio:
                    min_index = it
                    min_ratio = T[it,-1]/T[it,j] 
        
        # i = np.argmin(T[:-1,-1]/T[:-1,j]) #Doubt about correctness of this
        
        i = min_index
        
        if min_index == -1:
            # unbounded solution
            return None, None,None
        
        
        # step 3: perform pivot
        basis[i] = j
        
        T[i,:] /= T[i,j]
        for k in range(m+1):
            if k != i:
                T[k,:] -= T[k,j] * T[i,:]


In [293]:
m, n_old = A.shape
n = n_old + m
# Converting to standard form Ax = b
A = np.hstack((A, np.eye(A.shape[0])))
c = np.hstack((c, np.zeros(A.shape[0])))
# Minimising c
c = -c

# making b positive
for i in range(m):
    if(b[i] < 0):
        b[i] = -b[i]
        A[i,:] = -A[i,:]

# Phase1
# Ax + Iy = b
c_phase1 = np.concatenate((np.zeros(n), np.ones(m)))
A_phase1 = np.hstack((A, np.eye(m)))
basis_indices = np.arange(n, n+m)

# Constructing initial tableau
T = np.zeros((m+1, n+m+1))
# T[1:, 1:] = A
# T[1:,0] = b
# T[0, 1:] = c_phase1 - np.ones(m).T @ A
# T[0,0] = -np.ones(m) @ b
T[:-1, :-1] = A_phase1
T[0:-1:,-1] = b
T[-1, :-1] = c_phase1 - np.ones(m).T @ A_phase1
T[-1,-1] = -np.ones(m) @ b

# Solving the initial tableau
T, basis_indices, x = tableau_phase1(T, n, m, basis_indices)

if(abs(T[-1,-1]) > tol):
    
    print("F Cost not 0 ")

T = np.hstack((T[:,:n],T[:,n+m:]))
c_b = c[basis_indices]
T[-1, :-1] = c - c_b @ T[:-1, :-1]
T[-1,-1] = -c_b @ T[0:-1:,-1]


T, basis_indices, x = tableau_phase2(T, n, m, basis_indices)

print(T)

while True:
    # Check if solution is all integers
    flag = 1
    if  x is None:
        print("fat gaya")
        break
    for it in range(len(x)):
        if abs(int(x[it]) - x[it]) > tol:
            flag = 0
            break
    if flag:
        print (x[:n_old].astype(int))
        print(T[-1,-1])
        break
    
    # Add another constraint
    r = x - np.floor(x)
    j = np.argmax(r)

    basis_indices = np.hstack((basis_indices, len(x)))
    b_gomory = T[:-1,-1]
    b_gomory = np.append(b_gomory, -(x[j] - np.floor(x[j])))
    A_gomory = T[:-1, :-1]
    a = np.zeros(n)
    tmp = T[j, :-1]
    for it in range(n):
        if it not in basis_indices and abs(int(tmp[it]) - tmp[it]) > tol:
            a[it] = -(tmp[it] - np.floor(tmp[it]))
    A_gomory = np.vstack([A_gomory, a])

    a = np.zeros(m+1)
    a[-1] = 1
    A_gomory = np.hstack((A_gomory, np.array([a]).T, np.array([b_gomory]).T))

    temp = np.hstack((T[-1, :-1], 0 ,T[-1, -1]))
    T = np.vstack((A_gomory, temp))
    print(T)
    print("Basis",basis_indices)

    m += 1
    n += 1

    T, basis_indices, x = dual_simplex(T, basis_indices)
    print("After Dual")
    print(basis_indices)
    print(T)
    print(x)

[[ 0.    1.    1.5   0.   -0.25  2.25]
 [ 0.    0.   -2.    1.    0.5   0.5 ]
 [ 1.    0.   -0.5   0.    0.25  2.75]
 [ 0.    0.    0.5  -0.    0.25  7.75]]
[[ 0.    1.    1.5   0.   -0.25  0.    2.25]
 [ 0.    0.   -2.    1.    0.5   0.    0.5 ]
 [ 1.    0.   -0.5   0.    0.25  0.    2.75]
 [ 0.    0.   -0.5   0.   -0.75  1.   -0.75]
 [ 0.    0.    0.5  -0.    0.25  0.    7.75]]
Basis [1 3 0 5]
Entered Dual Simplex
[[ 0.00000000e+00  1.00000000e+00  1.66666667e+00  0.00000000e+00
   0.00000000e+00 -3.33333333e-01  0.00000000e+00  2.50000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.33333333e+00  1.00000000e+00
   0.00000000e+00  6.66666667e-01  0.00000000e+00  9.99200722e-16]
 [ 1.00000000e+00  0.00000000e+00 -6.66666667e-01  0.00000000e+00
   0.00000000e+00  3.33333333e-01  0.00000000e+00  2.50000000e+00]
 [-0.00000000e+00 -0.00000000e+00  6.66666667e-01 -0.00000000e+00
   1.00000000e+00 -1.33333333e+00  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -6.66666667e-0

In [294]:
b_gomory.shape

(9,)

In [295]:
A_gomory.shape

(9, 12)

In [296]:
a.shape

(9,)

In [297]:
x

In [298]:
x

In [299]:
basis_indices

In [300]:
A = np.ones((10,3))

In [301]:
b @ A

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10 is different from 3)

In [ ]:
b@b

10.0

In [ ]:
a = np.array([1,2,3])

In [ ]:
c = np.array([3,4,0])

In [ ]:
a/c

/tmp/ipykernel_750889/749814572.py:1: RuntimeWarning: divide by zero encountered in true_divide
  a/c


array([0.33333333, 0.5       ,        inf])

In [ ]:
basis = np.array([3,2,0])
a = np.array([7,1,2,3,4,5,6,7,8])
a[basis]

array([3, 2, 7])

In [ ]:
a < 3

array([False,  True,  True, False, False, False, False, False, False])

In [ ]:
a[a<3]

array([1, 2])

In [ ]:
x = 4.000000000
int(x) == x

False

In [ ]:
z = np.array([0.1,0.2])
np.sum(z)

0.30000000000000004

In [ ]:
# z.append(1)
np.hstack((z, 1))

array([0.1, 0.2, 1. ])

In [ ]:
np.array([z])

array([[0.1, 0.2]])

In [ ]:
np.array([z]).shape

(1, 2)